# Semantic Segmentation

No Ng lecture, but Stanford's deep learning course has an [excellent video lecture on this topic](https://www.youtube.com/watch?v=nDPWywWRIRo).  Please watch 00:08:00 - 00:33:00 or so (or keep watching to the end: it's pretty interesting stuff).  

So far, we've been concerned with the problem of object recognition: a mapping from an image to a class (or a probability of membership in each of N disjoint classes).  This is a useful thing as it allows us to determine the subject in an image.  It's also highly reductive, in the sense that we throw away all of the information about where in the image the object in question was located.  We were able to recover this information in some sense through class activation mapping (CAM).  However, it's important to think about the question that class activation mapping is answering: *What was the neural network looking at when it made its decision?*.  Thus, if one was looking at a picture of a ship, the network may have been looking at a flat horizon, or the ocean surface and saying *the only class that shows up on the ocean is a ship, so I'm going to classify this image as a ship.* Another case that I noticed was when classifying deer, the class activation map often indicated that the void space between the deer's legs was a focus of attention.  Or take the following example, in which CAM hones in on a blurry representation of the face of both the horse and the dog: clearly, the other pixels composing the creature were not considered evidence.

<img src=cam.png />

An alternative and more explicit question that we can ask is: *what pixels in this image belong to class k?* As it turns out, we can code that question explicitly as a neural network.  

What should be the inputs and outputs of this neural network?  Of course, the input is the same as for object recognition: usually a 3-band color image.  However, now instead of an array of probabilities for the entire image, we will have *an array of probabilities for each pixel*, thus our output image should be an image with the same width and height as the input, with N bands, where each band contains the probability of class membership.  Note that for the problem that we will address in this section, we will only use two classes (member of target class versus not member).  When this is the case, the output needs to only be one band: probability of membership.  Since these classes are disjoint, we can subtract the probability of membership from one to get the probability of not-membership.  Thus, we will be taking a color image, and outputting for each pixel the probability that it is a member of our target class.  

First, let




